In [1]:
from scapy.all import *
import csv
import os
import pandas as pd
import numpy as np

In [2]:
output_csv_file = 'agg_flow_data.csv'

In [3]:
dataset_path = 'dataset'
try:
    os.remove(output_csv_file)
    print(f"{output_csv_file} has been successfully deleted.")
except FileNotFoundError:
    print(f"{output_csv_file} does not exist.")

for filename in os.listdir(dataset_path):
    if 'pcap' in filename:
        pcap_file = os.path.join(dataset_path, filename)
        # print(pcap_file)
        print(pcap_file)

agg_flow_data.csv does not exist.
dataset/nonvpn_skype-chat_capture18.pcap
dataset/nonvpn_skype-chat_capture6.pcap
dataset/regular_browsing.pcap
dataset/nonvpn_skype-chat_capture14.pcap
dataset/nonvpn_skype-chat_capture39.pcap
dataset/nonvpn_skype-chat_capture15.pcap
dataset/nonvpn_skype-chat_capture7.pcap
dataset/nonvpn_skype-chat_capture19.pcap
dataset/nonvpn_skype-chat_capture12.pcap
dataset/nonvpn_netflix_capture1.pcap
dataset/regular_browsing1.pcap
dataset/nonvpn_skype-chat_capture1.pcap
dataset/nonvpn_skype-chat_capture13.pcap
dataset/nonvpn_skype-chat_capture10.pcap
dataset/regular_browsing2.pcap
dataset/nonvpn_skype-chat_capture2.pcap
dataset/nonvpn_skype-chat_capture3.pcap
dataset/nonvpn_skype-chat_capture11.pcap
dataset/vpn_youtube_capture1.pcap
dataset/nonvpn_skype-chat_capture16.pcap
dataset/nonvpn_skype-chat_capture8.pcap
dataset/nonvpn_skype-chat_capture9.pcap
dataset/nonvpn_sftp_capture2.pcap
dataset/nonvpn_skype-chat_capture5.pcap
dataset/vpn_skype-chat_capture6.pcap
da

In [17]:
def get_packet_details(pcap_file):
    packets=rdpcap(pcap_file)
    if 'netflix' in pcap_file or 'youtube' in pcap_file:
        label = 'video_stream'
    elif 'skype' in pcap_file:
        label = 'messaging'
    elif 'sftp' in pcap_file:
        label = 'file_transfer'
    elif 'regular_browsing' in pcap_file:
        label = 'regular_browsing'
    else:
        print(pcap_file)
        
    print(label,pcap_file, len(packets))

    packet_list = []
    
    for packet in packets:
        if IP in packet:
            packet_info = {
                'label': label,
                'timestamp': packet.time,
                'source_ip': packet[IP].src,
                'destination_ip': packet[IP].dst,
                'protocol': packet[IP].proto,
                'source_port': packet[IP].sport if TCP in packet or UDP in packet else None,
                'destination_port': packet[IP].dport if TCP in packet or UDP in packet else None,
                'packet_length': packet[IP].len,
                'label' : label
            }
            packet_list.append(packet_info)
    
    df = pd.DataFrame(packet_list)
    df = df.sort_values(by='timestamp')
    df['timestamp'] = df['timestamp'].astype(float)
    # Convert timestamps to a datetime format for easier manipulation
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    
    # Calculate time differences to find inter-arrival times
    df['time_diff'] = df['timestamp'].diff().dt.total_seconds().fillna(0)
    
    # Group by 0.25 second intervals. Assuming the first timestamp is the start.
    df['time_interval'] = (df['timestamp'] - df['timestamp'].min()).dt.total_seconds() // 0.25
    
    # Group by 'time_interval' and calculate the required attributes
    aggregated_data = df.groupby('time_interval').agg(
        number_of_packets=('timestamp', 'size'),
        average_packet_length=('packet_length', lambda x: round(x.mean(), 2)),
        number_of_unique_dests=('destination_ip', 'nunique'),
        TCP_protocol_count=('protocol', lambda x: (x==6).sum()),
        UDP_protocol_count=('protocol', lambda x: (x==17).sum()),
        #Inter_arrival_time_variance=('time_diff', np.var),
        #Packet_length_variance=('packet_length', np.var)
        #Inter_arrival_time_variance=('time_diff', lambda x: np.var(x, ddof=1)),
        #Packet_length_variance=('packet_length', lambda x: np.var(x, ddof=1))
        Packet_length_variance=('packet_length', lambda x: round(x.var() if len(x) > 1 else 0, 2)),
        Inter_arrival_time_variance=('time_diff', lambda x: round(x.var() if len(x) > 1 else 0, 2)),
        label = ('label','first')
    ).reset_index(drop=True)
    
    return aggregated_data
    

In [21]:
dataset_path = 'dataset'
try:
    os.remove(output_csv_file)
    print(f"{output_csv_file} has been successfully deleted.")
except FileNotFoundError:
    print(f"{output_csv_file} does not exist.")
    
final_df =  pd.DataFrame()

for filename in os.listdir(dataset_path):
    if 'pcap' in filename:
        pcap_file = os.path.join(dataset_path, filename)
        # print(pcap_file)
        print(pcap_file)
        packet_df = get_packet_details(pcap_file)
        final_df = pd.concat([final_df, packet_df], ignore_index=True)

agg_flow_data.csv does not exist.
dataset/nonvpn_skype-chat_capture18.pcap
messaging dataset/nonvpn_skype-chat_capture18.pcap 5540
dataset/nonvpn_skype-chat_capture6.pcap
messaging dataset/nonvpn_skype-chat_capture6.pcap 5728
dataset/nonvpn_skype-chat_capture14.pcap
messaging dataset/nonvpn_skype-chat_capture14.pcap 5672
dataset/nonvpn_skype-chat_capture39.pcap
messaging dataset/nonvpn_skype-chat_capture39.pcap 5969
dataset/nonvpn_skype-chat_capture15.pcap
messaging dataset/nonvpn_skype-chat_capture15.pcap 5490
dataset/nonvpn_skype-chat_capture7.pcap
messaging dataset/nonvpn_skype-chat_capture7.pcap 5476
dataset/nonvpn_skype-chat_capture19.pcap
messaging dataset/nonvpn_skype-chat_capture19.pcap 5780
dataset/nonvpn_skype-chat_capture12.pcap
messaging dataset/nonvpn_skype-chat_capture12.pcap 5921
dataset/nonvpn_netflix_capture1.pcap
video_stream dataset/nonvpn_netflix_capture1.pcap 240855
dataset/regular_browsing1.pcap
regular_browsing dataset/regular_browsing1.pcap 82574
dataset/nonvpn_

In [22]:
final_df.to_csv(output_csv_file, index=False)


In [23]:
print(len(final_df))

44270
